In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

# 手动设置环境变量
# os.environ["OPENAI_API_BASE_URL"] = "https://ai-yyds.com/v1"
# os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXX0bDb"
# os.environ["MODEL_NAME"] = "test"

## 记忆（Memory）工具
LangChain 提供了多种记忆（Memory）工具，用于在对话或链的执行过程中保持状态。这些工具可以帮助语言模型（LLM）记住之前的交互，从而生成更连贯和上下文相关的响应。

## 不同的Memory工具
利用内存实现短时记忆
利用Entity memory构建实体记忆
利用知识图谱来构建记忆
利用对话摘要来兼容内存中的长对话
使用token来刷新内存缓冲区
使用向量数据库实现长时记忆

### 基于内存实现短时记忆

#### ConversationBufferMemory
ConversationBufferMemory 是最简单的记忆形式，它将所有对话消息存储在一个缓冲区中。
它保留了完整的对话历史记录。

In [ ]:
from langchain.memory import ConversationBufferMemory

# 创建一个 ConversationBufferMemory 实例
memory = ConversationBufferMemory()

# 添加对话消息
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

# 从内存中加载记忆变量
memory.load_memory_variables({})

#### ConversationBufferWindowMemory 
ConversationBufferWindowMemory只保留最近的 k 个交互。
它适用于需要保留有限上下文的情况。

In [ ]:
# 实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import ConversationBufferWindowMemory

# 创建一个 ConversationBufferWindowMemory 实例，并指定 k 的值为 2。
memory = ConversationBufferWindowMemory(k=2)

# 将当前对话的上下文保存到内存中
memory.save_context({"input":"你好，我是人类!"}, {"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"}, {"output":"好的，我帮你找找鸡肉的做法"})

# 从内存中加载记忆变量
memory.load_memory_variables({})

#### 构建记忆实体概念清单
ConversationEntityMemory 是一种专门用于跟踪对话中实体的记忆工具。它不仅能记住对话的历史，还能提取和总结对话中提到的实体，并随着对话的进行不断更新这些实体的信息。
+ 功能：
  + 实体跟踪：
    + ConversationEntityMemory 的主要功能是识别和跟踪对话中出现的实体。
    + 它使用语言模型（LLM）来提取实体信息，并随着对话的进行更新这些信息。
  + 上下文提供：
    + 通过维护实体信息，ConversationEntityMemory 可以为 LLM 提供更丰富的上下文，从而生成更连贯和相关的响应。
+ 工作原理：
  + 实体提取：
    + 在对话过程中，ConversationEntityMemory 使用 LLM 从对话文本中提取实体。
  + 实体存储：
    + 提取出的实体及其相关信息被存储在内存中。
  + 实体更新：
    + 随着对话的进行，ConversationEntityMemory 会不断更新已存储的实体信息，以反映对话的最新状态。
  + 上下文提供：
    + 当 LLM 需要生成响应时，ConversationEntityMemory 会将相关的实体信息作为上下文提供给 LLM。
+ 使用场景：
  + 需要跟踪特定实体的对话：
    + 例如，在客户服务对话中，跟踪客户的订单信息、产品信息等。
  + 需要维护实体状态的对话：
    + 例如，在虚拟助手对话中，跟踪用户的偏好、日程安排等。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationEntityMemory
import os

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 初始化实体内存
memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}

# 从对话的内容中提取实体概念
# memory.load_memory_variables(_input)

# 写入记忆中
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)

In [ ]:
memory.load_memory_variables({"input":"铁三角是谁?"})

#### 使用知识图谱构建记忆
ConversationKGMemory 使用知识图谱（Knowledge Graph）来存储对话信息。
它可以更好地理解实体之间的关系。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 创建一个 ConversationKGMemory 实例，并指定用于知识图谱操作的 LLM
memory = ConversationKGMemory(llm=llm, return_messages=True)

# 添加一组对话记忆
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

# 添加一组对话记忆
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [ ]:
memory.load_memory_variables({"input":"tomie是谁?"})

In [ ]:
# 提取实体概念
memory.get_current_entities("tomie最喜欢做什么事?")

In [ ]:
# 获取三元知识组
memory.get_knowledge_triplets("tomie最喜欢打游戏")

#### 长对话在内存中的处理方式：总结摘要以及token计算
在 LangChain 中，ConversationSummaryMemory 是一种专门用于处理长对话的记忆工具。它通过总结对话的早期部分，只保留最近的交互，从而有效地管理对话的上下文，避免因对话过长而超出语言模型（LLM）的上下文窗口限制。
在 LangChain 中，ConversationSummaryMemory 是一种专门用于处理长对话的记忆工具。它通过总结对话的早期部分，只保留最近的交互，从而有效地管理对话的上下文，避免因对话过长而超出语言模型（LLM）的上下文窗口限制。

以下是关于 ConversationSummaryMemory 的详细解释：
+ 功能：
  + 对话摘要：
    + ConversationSummaryMemory 的主要功能是总结对话的早期部分，将长对话压缩成更短的摘要。
    + 它使用 LLM 来生成对话的摘要。
  + 上下文管理：
    + 通过维护对话摘要，ConversationSummaryMemory 可以为 LLM 提供更精简的上下文，从而生成更连贯和相关的响应。
    + 它有效地解决了长对话中上下文窗口限制的问题。
+ 工作原理：
  + 对话存储：
    + ConversationSummaryMemory 像 ConversationBufferMemory 一样，存储对话的完整历史记录。
  + 摘要生成：
    + 当对话长度超过一定阈值时，ConversationSummaryMemory 使用 LLM 生成对话的摘要。
  + 上下文提供：
    + 在后续的对话中，ConversationSummaryMemory 将摘要和最近的交互作为上下文提供给 LLM。
+ 使用场景：
  + 长对话：
    + 当对话可能很长，并且需要保留对话的整体上下文时。
  + 资源有限：
    + 当计算资源有限，无法存储完整的对话历史记录时。

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [ ]:
# 查看所有的记忆内容
memory.load_memory_variables({})

In [ ]:
# 获取完整的对话内容
messages = memory.chat_memory.messages
# print(messages)

# 产生对话摘要
memory.predict_new_summary(messages, "")

In [ ]:
# 使用ChatMessageHistory来快速获得对话摘要
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 定义历史对话记录
history = ChatMessageHistory()
history.add_user_message("你好，我是人类！")
history.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

memory = ConversationSummaryMemory.from_messages(
   llm=llm,
   chat_memory=history,
   return_messages=True
)

# 方式二：在初始化中把对话放入buffer中
# memory = ConversationSummaryMemory(
#     llm=llm,
#     return_messages=True,
#     buffer="\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.",
#     chat_memory=hisiory
# )

In [ ]:
# 查看内存中的内容 memory.buffer
memory.load_memory_variables({})

#### ConversationSummaryBufferMemory
当对话持续进行且对话内容很多的时候，可以使用ConversationSummaryBufferMemory来存储对话摘要
ConversationSummaryBufferMemory是一种结合了ConversationBufferMemory和ConversationSummaryMemory特点的记忆工具，它的目的是在处理长对话时，既保持对话的连贯性，又避免因对话过长而超出语言模型（LLM）的上下文窗口限制。
这是一种非常有用的方式，它会根据token的数量来自动判断是否需要进行摘要
当token数量超过阈值的时候，会自动进行摘要
在缓冲区中，会保留最近的k条对话
比较久的对话会被删除，在删除前会进行摘要

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    # 设置触发摘要生成的最大令牌数限制。可以根据LLM的上下文窗口的大小进行调整。
    max_token_limit=10,
    return_messages=True
)

# 添加对话内容
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)


In [ ]:
memory.load_memory_variables({})

#### ConversationTokenBufferMemory使用token长度来决定什么时候刷新内存
在 LangChain 中，ConversationTokenBufferMemory 是一种用于管理对话历史记录的记忆工具，它与 ConversationBufferMemory 类似，但它使用令牌（tokens）的数量而不是消息的数量来确定何时截断对话历史记录。

以下是关于 ConversationTokenBufferMemory 的详细解释：

+ 功能：
  + 令牌缓冲区：
    + ConversationTokenBufferMemory 将对话历史记录存储在一个缓冲区中，但它会跟踪对话中使用的令牌数量。
    + 当令牌数量超过指定的限制时，它会从缓冲区中删除最早的消息，以保持在限制范围内。
  + 上下文管理：
    + 通过限制对话历史记录的令牌数量，ConversationTokenBufferMemory 可以有效地管理对话的上下文，避免因对话过长而超出语言模型（LLM）的上下文窗口限制。

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

memory = ConversationTokenBufferMemory(
    # 指定用于令牌计数的 LLM
    llm=llm,
    # 设置最大令牌数限制
    max_token_limit=50
)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

In [ ]:
memory.load_memory_variables({})

### 长时记忆的是实现方式

通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

memory = ConversationBufferMemory()

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

# 定义向量数据库
vector_store = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)

# 将向量数据库存在本地文件
FAISS.save_local(vector_store, "test_faiss")

In [ ]:
# 进行相似性搜索
FAISS.load_local("test_faiss", 
                 OpenAIEmbeddings(model="text-embedding-3-small"), 
                 allow_dangerous_deserialization=True).similarity_search("tomie是什么职业?")

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

# 加载长期记忆文件
r1 = FAISS.load_local("test_faiss", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})